POSTagging dengan metode baseline:
1. kata yang sudah pernah muncul di data latih akan diberi label POSTag kata tersebut dengan frekuensi kemunculan tertinggi
2. kata yang belum pernah muncul di data latih akan diberi label POSTag yang paling sering muncul di data latih

Import library yang dibutuhkan

In [3]:
import nltk
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Baca dataset

In [4]:
colnames = ['Word','Tag']
tsv_read = pd.read_csv('train.01.tsv', sep='\t',names=colnames,header=None)
tsv_read = tsv_read.astype(str)
tsv_read.head()

,Word,Tag
0,Pemerintah,NN
1,bahkan,RB
2,telah,MD
3,mencanangkan,VB
4,dana,NN


In [5]:
word_tag_freq = {} # dictionary untuk menyimpan frekuensi tag kata
tag_count = {} # dictionary untuk menyimpan frekuensi tag
count_sentences = 0

for index, row in tsv_read.iterrows():
    word = row['Word'].lower()
    tag = row['Tag']    
    key = (word,tag) # simpan di dictionary kata,tag
    # print(word,tag)
    if count_sentences < 50:
        # simpan di dictionary tag
        if tag in tag_count:
            tag_count[tag] = tag_count[tag] + 1
        else:
            tag_count[tag] = 1
            
        if key in word_tag_freq:
            word_tag_freq[key] = word_tag_freq[key] + 1
        else:
            word_tag_freq[key] = 1

    if word == '.' :
        count_sentences += 1      
        if(count_sentences == 50):
            break

In [6]:
print(word_tag_freq)

{('pemerintah', 'NN'): 6, ('bahkan', 'RB'): 1, ('telah', 'MD'): 6, ('mencanangkan', 'VB'): 1, ('dana', 'NN'): 1, ('untuk', 'SC'): 9, ('memicu', 'VB'): 1, ('sektor', 'NN'): 2, ('usaha', 'NN'): 1, ('kecil', 'JJ'): 2, ('dan', 'CC'): 19, ('menengah', 'JJ'): 1, ('ukm', 'NN'): 1, ('tumbuh', 'VB'): 1, ('lebih', 'RB'): 3, ('baik', 'JJ'): 2, (',', 'Z'): 62, ('karena', 'SC'): 7, ('ini', 'PR'): 8, ('cukup', 'RB'): 2, ('kuat', 'JJ'): 2, ('dalam', 'IN'): 13, ('krisis', 'JJ'): 1, ('keuangan', 'NN'): 2, ('pada', 'IN'): 13, ('tahun', 'NN'): 9, ('1997', 'CD'): 1, ('lalu', 'CC'): 3, ('kata', 'VB'): 5, ('-nya', 'PRP'): 21, ('.', 'Z'): 48, ('nan', 'nan'): 49, ('gubernur', 'NNP'): 1, ('bank', 'NNP'): 6, ('indonesia', 'NNP'): 11, ('burhanudin', 'NNP'): 1, ('abdullah', 'NNP'): 1, ('optimistis', 'JJ'): 1, ('mencapai', 'VB'): 6, ('target', 'NN'): 3, ('proyeksi', 'NN'): 1, ('suku', 'NN'): 2, ('bunga', 'NN'): 4, ('sbi', 'NNP'): 1, ('2008', 'CD'): 2, ('yang', 'SC'): 28, ('7,5', 'CD'): 1, ('-', 'Z'): 2, ('8,0', 'C

In [7]:
print(tag_count)

{'NN': 330, 'RB': 24, 'MD': 31, 'VB': 151, 'SC': 67, 'JJ': 57, 'CC': 37, 'Z': 115, 'PR': 31, 'IN': 115, 'CD': 94, 'PRP': 31, 'nan': 49, 'NNP': 174, 'NND': 3, 'SYM': 16, 'OD': 4, 'X': 1, 'NEG': 8, 'FW': 8, 'DT': 1}


Buat fungsi untuk mengetahui tag yang paling sering muncul di data latih

In [8]:
def get_most_freq_tag():
    most_freq_tag = max(tag_count, key=tag_count.get) 
    return most_freq_tag

Tes fungsi mengambil tag yang paling sering muncul di data latih

In [9]:
default_tag = get_most_freq_tag()
print(default_tag)

NN


Buat fungsi untuk mencari tag yang paling sering muncul pada sebuah kata

In [10]:
def get_most_freq_tag_word(word):
    word_tags = {k: v for k, v in word_tag_freq.items() if word in k}
    sorted_word_tags = sorted(word_tags.items(), key=lambda x: x[1], reverse=True)
    if len(sorted_word_tags)>0:
        return sorted_word_tags[0]
    else:
        return None

Tes fungsi mencari tag yang paling sering muncul pada sebuah kata

In [23]:
word_tags = get_most_freq_tag_word('bisa')
if word_tags:
    print('informasi tentang tag kata yang paling sering muncul')
    print('kata:',word_tags[0][0],', tag:',word_tags[0][1],', frekuensi:',word_tags[1])
else:
    print('kata tidak ditemukan di data latih')

informasi tentang tag kata yang paling sering muncul
kata: bisa , tag: MD , frekuensi: 1


Tes pada kalimat uji

In [12]:
colnames = ['Word','Tag']
tsv_read_test = pd.read_csv("test_sentences.tsv",sep="\t", names=colnames, header=None)
tsv_read_test = tsv_read_test.astype(str)
tsv_read_test.head()

,Word,Tag
0,Menteri,NNP
1,pertahanan,NNP
2,AS,NNP
3,dijadwalkan,VB
4,mengunjungi,VB


In [13]:
testing_sentence = [] 
word_temp = [] 
label_sentences = []
sentences = [] 

for index, row in tsv_read_test.iterrows():   
    word = row['Word'].lower()
    tag = row['Tag']
    if word != 'nan':
      testing_sentence.append(word)
      word_temp.append(word)
      label_sentences.append(tag)
      if word == '.' :
        sentences.append(' '.join(word_temp))
        word_temp = []
        tags = []

In [14]:
print(sentences)

['menteri pertahanan as dijadwalkan mengunjungi india .', 'tata power menyuplai batu bara pada tahun 2000 .', 'pemerintah hati-hati dalam mengelola bumn .', 'perusahaan baru tersebut mencanangkan target perolehan laba bersih .', 'menteri pertahanan mengunjungi pangkalan udara .', 'menurut laporan sekretaris perusahaan , laba bersih meningkat .', 'transaksi penjualan barang mewah tahun 2007 turun .', 'menkeu memperkirakan inflasi akan meningkat dibanding tahun lalu .', 'kenaikan tarif didorong oleh target laba bersih yang meningkat .', 'makanan dari luar negeri tidak bisa masuk pasar lokal .']


Hasil tagging pada kalimat uji

In [15]:
tagging = []

for token in testing_sentence:
    most_freq_tag_token = get_most_freq_tag_word(token.lower())
    if most_freq_tag_token:
        print('Word:',token,', Tag:',most_freq_tag_token[0][1])
        tagging.append(most_freq_tag_token[0][1])
    else:
        print('Word:',token,' tidak terdapat di data latih, Tag:',get_most_freq_tag())
        tagging.append(get_most_freq_tag())

Word: menteri , Tag: NNP
Word: pertahanan , Tag: NN
Word: as , Tag: NNP
Word: dijadwalkan , Tag: VB
Word: mengunjungi , Tag: VB
Word: india , Tag: NNP
Word: . , Tag: Z
Word: tata , Tag: NNP
Word: power , Tag: NNP
Word: menyuplai , Tag: VB
Word: batu bara , Tag: NN
Word: pada , Tag: IN
Word: tahun , Tag: NN
Word: 2000 , Tag: CD
Word: . , Tag: Z
Word: pemerintah , Tag: NN
Word: hati-hati , Tag: JJ
Word: dalam , Tag: IN
Word: mengelola , Tag: VB
Word: bumn , Tag: NN
Word: . , Tag: Z
Word: perusahaan , Tag: NN
Word: baru , Tag: JJ
Word: tersebut , Tag: PR
Word: mencanangkan , Tag: VB
Word: target , Tag: NN
Word: perolehan , Tag: NN
Word: laba bersih , Tag: NN
Word: . , Tag: Z
Word: menteri , Tag: NNP
Word: pertahanan , Tag: NN
Word: mengunjungi , Tag: VB
Word: pangkalan , Tag: NN
Word: udara , Tag: NN
Word: . , Tag: Z
Word: menurut , Tag: IN
Word: laporan , Tag: NN
Word: sekretaris , Tag: NNP
Word: perusahaan , Tag: NN
Word: , , Tag: Z
Word: laba bersih , Tag: NN
Word: meningkat , Tag: VB


In [18]:
print(tagging)

['NNP', 'NN', 'NNP', 'VB', 'VB', 'NNP', 'Z', 'NNP', 'NNP', 'VB', 'NN', 'IN', 'NN', 'CD', 'Z', 'NN', 'JJ', 'IN', 'VB', 'NN', 'Z', 'NN', 'JJ', 'PR', 'VB', 'NN', 'NN', 'NN', 'Z', 'NNP', 'NN', 'VB', 'NN', 'NN', 'Z', 'IN', 'NN', 'NNP', 'NN', 'Z', 'NN', 'VB', 'Z', 'NN', 'NN', 'NN', 'NN', 'CD', 'VB', 'Z', 'NNP', 'VB', 'NN', 'MD', 'VB', 'VB', 'NN', 'CC', 'Z', 'NN', 'NN', 'VB', 'IN', 'NN', 'NN', 'SC', 'VB', 'Z', 'NN', 'IN', 'NN', 'NEG', 'MD', 'VB', 'NN', 'JJ', 'Z']


In [22]:
count = 0
for i in range(len(tagging)):
    if label_sentences[i] == tagging[i]:
        count += 1
    else:
        count += 0

print('Akurasi dengan metode Baseline: ', count/len(tagging))

Akurasi dengan metode Baseline:  0.948051948051948
